# Scraping SEC Website for Financials

In [1]:
import edgar
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup

In [2]:
#assuming we already have this information
#edgar.download_index("C:\\Users\\605089\\Desktop\\Personal Projects\\Financial", 2011, "your_name your@email.address", skip_all_present_except_last=False)

## Functions

In [3]:
def find_10k_quarter(selected_company, selected_report, year):
    quarters = ['QTR1', 'QTR2', 'QTR3', 'QTR4']
    
    master_index_file = ''
        
    for quarter in quarters:
        
        master_index_file = str(year) + '-' + quarter + '.tsv'
        
        master_index = pd.read_csv(master_index_file, sep='\t', lineterminator='\n', names=None)
        
        master_index.columns.values[0] = 'Item'
    
        filing_index = master_index[(master_index['Item'].str.contains(selected_company, case=False)) & (master_index['Item'].str.contains(selected_report, case=False))]
        
        if not filing_index.empty:
            return quarter
    
    return ''

In [4]:
def get_index_info_on_company(selectedcompany, selectedreport, master_index_file):
    #read master index info into a pandas dataframe
    master_index = pd.read_csv(master_index_file, sep='\t', lineterminator='\n', names=None)

    #rename column for convenience
    master_index.columns.values[0] = 'Item'

    #select the row in which the company and document you are interested in is located
    filing_index = master_index[(master_index['Item'].str.contains(selectedcompany, case = False)) & (master_index['Item'].str.contains(selectedreport, case=False))]

    filing_index = filing_index['Item'].str.split('|').to_list()

    for elem in filing_index:
        if selectedreport in elem:
            filing_index_clean = elem
            
    print(filing_index_clean)
            
    return filing_index_clean
    

The below functions gets the xml file of a specific filing using Edgar index files downloaded from the edgar python library. Then parses through the xml summary of the filing to obtain the specific urls of various sections of the filing.

In [5]:
def get_xml_summary_of_file(selectedcompany, selectedreport, master_index_file, user_agent):
    #read master index info into a pandas dataframe
    master_index = pd.read_csv(master_index_file, sep='\t', lineterminator='\n', names=None)

    #rename column for convenience
    master_index.columns.values[0] = 'Item'

    #select the row in which the company and document you are interested in is located
    filing_index = master_index[(master_index['Item'].str.contains(selectedcompany, case=False)) & (master_index['Item'].str.contains(selectedreport, case=False))]

    filing_index = filing_index['Item'].str.split('|').to_list()

    for elem in filing_index:
        if selectedreport in elem:
            filing_index = elem

    for item in filing_index:
        if '.txt' in item:
             txt_url = item
                
    #define base url
    base_url = r"https://www.sec.gov"

    #convert the txt_url to a json url for the document landing page
    documents_url = base_url + "/Archives/" + txt_url.replace('-','').replace('.txt', '/index.json')

    content = requests.get(documents_url, headers=user_agent).json()


    #for each file in the document landing page
    for file in content['directory']['item']:
    
        #if the filing's name is FilingSummary.xml
        if file['name'] == 'FilingSummary.xml':
        
            #create the url for the xml file
            xml_summary = base_url + content['directory']['name'] + "/" + file['name']
        
            print('-' * 100)
            print('File Name: ' + file['name'])
            print('File Path: ' + xml_summary)
        
    return xml_summary

In [6]:
def parse_xml_summary_of_file(xml_summary, user_agent):
    #define a new base url that represents the filing folder. This will come in handy when we need to download the reports
    base_url = xml_summary.replace('FilingSummary.xml', '')

    #request and parse the content (use .content because this is an xml file not json)
    content = requests.get(xml_summary, headers=user_agent).content

    #create a soup object to parse the content using BeautifulSoup
    soup = BeautifulSoup(content, 'lxml')

    #find the 'myreports' tag because this contains all the individual reports submitted
    reports = soup.find('myreports')

    #want a list ot store all theindividual components of the report, so create the master list
    master_reports = []

    # loop through each report in the 'myreports' tag but avoid the last one as this will cause an error
    # (last one has differnent formatting).
    for index, report in enumerate(reports.find_all('report')[:-1]):
    
        #create a dictionary to store all the differnet parts we need
        report_dict = {}
        report_dict['name_short'] = report.shortname.text
        report_dict['name_long'] = report.longname.text
        report_dict['position'] = index+1
        #report_dict['category'] = report.menucategory.text
        report_dict['url'] = base_url + report.htmlfilename.text
    
        #append the dictionary to the master list.
        master_reports.append(report_dict)
    
        #print the info to the user.
        print('-'*100)
        print(base_url + report.htmlfilename.text)
        print(report.longname.text)
        print(report.shortname.text)
        #print(report.menucategory.text)
        print(index+1)
        
    return master_reports

Below tries to take a list of urls linking towards the tables in a financial filing, select which urls we want to request content from, and then parse the html text to get specific table information from those urls.

In [7]:

##################### REVIST LATER #########################################################

def grab_financial_statements(master_reports):
    #create the list to hold the statement urls
    statements_url = []

    for report_dict in master_reports:
    
    # define the statements we want to look for.   
    #######################################################################################
    # MAY WANT TO REVISIT THIS
    #######################################################################################
        item1 = r"CONSOLIDATED"
        item2 = r"PARENTHETICAL"
        item3 = r"EQUITY"
    
        if item1 in report_dict['name_short'].upper() and item2 not in report_dict['name_short'].upper() and item3 not in report_dict['name_short'].upper() and report_dict['position'] < 12:
            statements_url.append(report_dict['url'])
        
        
    # NOW WE HAVE THE URLS OF THE STATEMENTS WE WANT TO SCRAPE
        
        
    #assuming we want all the statements in a single data set
    statements_data = []

    #loop through each statement url
    for statement in statements_url:
    
        #define a dictionary that will store the different parts of the statement
        statement_data = {}
        statement_data['headers'] = []
        statement_data['sections'] = []
        statement_data['data'] = []
    
        #request the statement file content
        content = requests.get(statement, headers=user_agent).content
        report_soup = BeautifulSoup(content, 'html')
    
        #find all the rows, figure out what type of row it is, parse the elements, and store in teh statement file list.
        #tr = table row
        for index, row in enumerate(report_soup.table.find_all('tr')):
        
            #first let's get all the elements (td = standard data cell)
            # cols represents elements in the row
            cols = row.find_all('td')
            
           # if it's a regular row and not a section or a table header (th = header, strong = important)
            if (len(row.find_all('th')) == 0 and len(row.find_all('table')) == 0): # and len(row.find_all('strong')) == 0:
                reg_row = []
                for ele in cols:
                
                    if len(row.find_all('strong')) != 0:
                        reg_row.append(ele.text.strip().upper())
                    else:
                        reg_row.append(ele.text.strip())
                        
                #reg_row = [ele.text.strip() for ele in cols]
                statement_data['data'].append(reg_row)
                        
                #check if it is a section label and store that it in sec_row
                if len(row.find_all('strong')) != 0:
                    sec_row = cols[0].text.strip().upper()
                    statement_data['sections'].append(sec_row)
                    
            
            #finally if it's not any of those it must be a header
            elif (len(row.find_all('th')) != 0):
                hed_row = []
                for ele in row.find_all('th'):
                    if (len(ele.find_all('sup'))) == 0:
                        hed_row.append(ele.text.strip())
                statement_data['headers'].append(hed_row)
            
            else:
                print("We encountered an error")
    
        statements_data.append(statement_data)
    
    return statements_data
    
    

Below are functions that deal with converting data into a pandas dataframe

In [8]:
## Purpose: Checks if a character can be converted into a float

def char_is_float(char):
    try:
        float(char)
        return True
    except:
        return False
    

## Purpose: Changes a (assumed) string into a float, getting rid of any non-number values in the string in the process.
## Assumptions: We are only interested in strings, so anything else will get a NaN value. In other words, we expect a string
##              but if otherwise then we ignore/erase the passed value as it was not expected and could cause further problems
##              if kept as is.
## Example: Converts '900f' to 900.0

def change_to_float(string):
    
    #if already a float, no need to continue
    if type(string) == float:
        return float(string)
    
    #check if the value is a string (since we want to strip the string and that doesn't work for other types)
    elif type(string) == str:
        
        #get rid of spaces
        string = string.strip()
        
        #just checking ... again
        if string == 'None':
            return 'NaN'
    
        #string to append float characters to
        string_to_convert = ""
        
        #iterate through each element of the string
        for i, char in enumerate(string):
            
            #check the first element of the string, as that can be a minus sign, otherwise only append chars that are numbers
            if i == 0:
                if char_is_float(char) or char == '-':
                    string_to_convert = string_to_convert + char
                
                #if the first character is not a number, why bother
                else:
                    return None
            else:
                if char_is_float(char):
                    string_to_convert = string_to_convert + char
            
        return float(string_to_convert)
    
    #if not a float or a string, I'm assuming its a Nonetype. Return NaN because that is float terminology
    else:
        return 'NaN'

In [9]:

## Purpose: Gets rid of any columns in a pandas datframe that do not contain any information (i.e. irrelevant)
def get_rid_of_null_columns(df):

    #store the column index of columns we want to drop in a list
    column_index_to_drop = []

    #for each column in the dataframe
    for i in range(df.shape[1]):
        
        #if all values within that column are NaN, assign that column to be dropped
        if df[df.columns[i]].isnull().all():
            column_index_to_drop.append(i)
        
    df.drop(df.columns[column_index_to_drop], axis = 1, inplace=True)

    return df




In [10]:
## Parameters: Takes in a dictionary called statements data that holds 'headers' and 'data' from the html that describes
##             the financial statements scraped from the SEC website. The 'headers' describe the columns, while the 'data'
##             are the elements below the 'headers' in the table. Statements data contains multiple documents, but this funciton
##             only works with one statement, so a number describing what statement from statements data is given as a 
##             parameter.
##
## Purpose:    Converts a dictionary describing financial data into a pandas dataframe. There are many steps to cleaning
##             the data so that it can be machine readable (e.g. cleaning the data, dealing with edge cases such as multiple
##             levels of headers)

def convert_statements_data_to_pandas(statements_data, statements_num):
    
    #take the raw table data (without the headers)
    balance_data = statements_data[statements_num]['data']
    
    df = pd.DataFrame(balance_data)
    
    #Define the Index column as the first column
    df.index = df[0]
    
    #since the index makes up the first column, we can get rid of the first column
    df = df.drop(0, axis = 1)
    
    # Get rid of the '$', '(', ')', and convert the '' to NaNs.
    df = df.replace('[\$,)]','', regex=True )\
                     .replace( '[(]','-', regex=True)\
                     .replace( '', 'NaN', regex=True)
    
    #convert each element in the table portion into a float (excluding indexes or headers) 
    for i in range(df.shape[1]):
        
        #apply change_to_float which cleans the element to be prepared to turn into a float
        df[df.columns[i]] = df[df.columns[i]].apply(change_to_float) 
    
    #change_to_float should already accomplish this, but this is to double check if any values were not converted into floats
    df = df.astype(float)
    
    df = get_rid_of_null_columns(df)
    
    
    # Now we need the headers for the document (as well as the document's name) 
    # This is also stored in the statements_data under headers
    
    # Because the headers may either have one or two levels in the financial statement (sometimes over the year, there is 
    # a statement like "12 months ended in"), we need to specify the headers that match up best with the columns
    
    # This may not make a whole bunch of sense but that's because getting the headers is specific to two different types
    # of headers for the financial documents.
    
    #if the headers only have one level (one row that holds all the headers)
    if len(statements_data[statements_num]['headers']) == 1:
        
        #headers get all the values in 'headers' except the first one which is used as the header for the index
        header = statements_data[statements_num]['headers'][0][1:]
        document = statements_data[statements_num]['headers'][0][0]
    
    #if the headers have two levels (there are two rows that hold all the headers)
    elif len(statements_data[statements_num]['headers']) == 2:
        
        #headers get the bottom row
        header = statements_data[statements_num]['headers'][-1]
        
        #document name gets first element of the top row
        document = statements_data[statements_num]['headers'][0][0] 
        
    #there are probably more edge cases, so we'll just keep everything the same to keep things moving
    else:
        header = df.columns
        document = 'Category'
    
    
    df.columns = header
    df.index.name = document
    
    return df
    

In [11]:
## Purpose: Some indices repeat in a financial statement (e.g. others). To make sure each index is unique without
##          losing information in the dataframe, we can add a tag to the end. The number of tags at the end represent
##          the number of times that index value has been seen before (no tag means first time, one tag means second, two
##          tags means third)
def add_tags_to_repeating_indices(df, tag):
    name = df.index.name
    idx = df.index.to_list()
    dup = {x for x in idx if idx.count(x) > 1}

    for ele in dup:
        end = ''
        for i in range(len(idx)):
            if ele == idx[i]:
                idx[i] = idx[i] + end
                end = end + tag
            
    df.index = idx
    df.index.name = name
    return df



In [12]:
def add_tags_for_sections(sections, df):
    if len(sections) < 2:
        return df
    idx = df.index.to_list()
    name = df.index.name
    
    idx_tags = []
    
    count = 0
    
    for ele in idx:
        if ele in sections:
            count = count + 1
        idx_tags.append(ele + '-' + str(count))
        
            
    df.index = idx_tags
    df.index.name = name
    
    return df
    

# Run Scrape

In [13]:
#look into master index file for precise spelling
#selectedcompany = 'Booz Allen Hamilton Holding Corp'

selectedcompany = r'Booz'

#get the CIK number (id for company)
#CIK_number = get_CIK_number(selectedcompany, selectedreport)

#select a year you are interested in
selectedyear = 2021
current_year = 2021

#make sure to put in report exact (10-K vs 10-k)
selectedreport = "10-K"

master_index_file = str(selectedyear) + '-' + find_10k_quarter(selectedcompany, selectedreport, selectedyear) + '.tsv'


#define a user agent for access to the SEC Edgar servers (use browser)
user_agent = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.62"}

print(selectedcompany)
print(master_index_file)


Booz
2021-QTR2.tsv


In [14]:
def scrape_SEC_for_statements(selectedcompany, selectedreport, year):
    master_index_file = str(year) + '-' + find_10k_quarter(selectedcompany, selectedreport, year) + '.tsv'
    filing_index = get_index_info_on_company(selectedcompany, selectedreport, master_index_file)
    xml_summary = get_xml_summary_of_file(selectedcompany, selectedreport, master_index_file, user_agent)
    master_reports = parse_xml_summary_of_file(xml_summary, user_agent)
    statements_data = grab_financial_statements(master_reports)
    
    dataframes = []
    for i in range(len(statements_data)):
        df = convert_statements_data_to_pandas(statements_data, i)
        df = add_tags_for_sections(statements_data[i]['sections'], df)
        df = add_tags_to_repeating_indices(df, '@')
        dataframes.append(df)
    return dataframes
    
    

In [15]:
## Purpose: Scrapes the SEC for financial information (specifically balance sheets, income statements, and cash flow 
##          statements) and places this information into dataframes. This function goes back a specified many years from
##          a starting "current" year
## 
## Return: A list containing a list of dataframes (financial statements) for a given year

def scrape_SEC_many_years(selectedcompany, selectedreport, current_year, num_years):
    
    # to store all the dataframes (will be a list (year) of lists (each financial statement) of dataframes (describing the statement))
    dfs = []

    # scrape the SEC for the past 10 years and store all that information in dataframes
    for i in range(num_years):
        year = current_year - i
    
        #scrape the SEC for statements, returns a list of dataframes
        temp_dataframes = scrape_SEC_for_statements(selectedcompany, selectedreport, year)
        
        #append the list of dataframes into a master list representing the year
        dfs.append(temp_dataframes)
    
    return dfs

In [16]:
#START SCRAPING

num_years = 10

dfs = scrape_SEC_many_years(selectedcompany, selectedreport, current_year, num_years)

#drop all the columns that do not pertain to the specific year of that dataframe

for i in range(len(dfs)):
    for j in range(len(dfs[i])):
        dfs[i][j].drop(columns=dfs[i][j].columns[1:], axis=1, inplace=True)


['1443646', 'Booz Allen Hamilton Holding Corp', '10-K', '2021-05-21', 'edgar/data/1443646/0001443646-21-000079.txt', 'edgar/data/1443646/0001443646-21-000079-index.html\r']
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/1443646/000144364621000079/FilingSummary.xml
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364621000079/R1.htm
0001001 - Document - Cover
Cover
1
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364621000079/R2.htm
1001002 - Statement - Consolidated Balance Sheets
Consolidated Balance Sheets
2
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/

['1443646', 'Booz Allen Hamilton Holding Corp', '10-K', '2020-05-26', 'edgar/data/1443646/0001443646-20-000086.txt', 'edgar/data/1443646/0001443646-20-000086-index.html\r']
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/1443646/000144364620000086/FilingSummary.xml
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364620000086/R1.htm
0001000 - Document - Cover
Cover
1
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364620000086/R2.htm
1001000 - Statement - Consolidated Balance Sheets
Consolidated Balance Sheets
2
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/

['1443646', 'Booz Allen Hamilton Holding Corp', '10-K', '2019-05-28', 'edgar/data/1443646/0001443646-19-000093.txt', 'edgar/data/1443646/0001443646-19-000093-index.html\r']
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/1443646/000144364619000093/FilingSummary.xml
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364619000093/R1.htm
0001000 - Document - Document and Entity Information
Document and Entity Information
1
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364619000093/R2.htm
1001000 - Statement - Consolidated Balance Sheets
Consolidated Balance Sheets
2
------------------------------------------------------------------------------------

['1443646', 'Booz Allen Hamilton Holding Corp', '10-K', '2018-05-29', 'edgar/data/1443646/0001443646-18-000100.txt', 'edgar/data/1443646/0001443646-18-000100-index.html\r']
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/1443646/000144364618000100/FilingSummary.xml
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364618000100/R1.htm
0001000 - Document - Document and Entity Information
Document and Entity Information
1
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364618000100/R2.htm
1001000 - Statement - Consolidated Balance Sheets
Consolidated Balance Sheets
2
------------------------------------------------------------------------------------

['1443646', 'Booz Allen Hamilton Holding Corp', '10-K', '2017-05-22', 'edgar/data/1443646/0001443646-17-000050.txt', 'edgar/data/1443646/0001443646-17-000050-index.html\r']
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/1443646/000144364617000050/FilingSummary.xml
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364617000050/R1.htm
0001000 - Document - Document and Entity Information
Document and Entity Information
1
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364617000050/R2.htm
1001000 - Statement - Consolidated Balance Sheets
Consolidated Balance Sheets
2
------------------------------------------------------------------------------------

['1443646', 'Booz Allen Hamilton Holding Corp', '10-K', '2016-05-19', 'edgar/data/1443646/0001443646-16-000138.txt', 'edgar/data/1443646/0001443646-16-000138-index.html\r']
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/1443646/000144364616000138/FilingSummary.xml
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364616000138/R1.htm
0001000 - Document - Document and Entity Information
Document and Entity Information
1
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364616000138/R2.htm
1001000 - Statement - Consolidated Balance Sheets
Consolidated Balance Sheets
2
------------------------------------------------------------------------------------

['1443646', 'Booz Allen Hamilton Holding Corp', '10-K', '2015-05-21', 'edgar/data/1443646/0001443646-15-000018.txt', 'edgar/data/1443646/0001443646-15-000018-index.html\r']
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/1443646/000144364615000018/FilingSummary.xml
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364615000018/R1.htm
0001000 - Document - Document and Entity Information
Document and Entity Information
1
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364615000018/R2.htm
1001000 - Statement - Consolidated Balance Sheets
Consolidated Balance Sheets
2
------------------------------------------------------------------------------------

['1443646', 'Booz Allen Hamilton Holding Corp', '10-K', '2014-05-22', 'edgar/data/1443646/0001443646-14-000015.txt', 'edgar/data/1443646/0001443646-14-000015-index.html\r']
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/1443646/000144364614000015/FilingSummary.xml
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364614000015/R1.htm
0001000 - Document - Document and Entity Information
Document and Entity Information
1
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364614000015/R2.htm
1001000 - Statement - Consolidated Balance Sheets
Consolidated Balance Sheets
2
------------------------------------------------------------------------------------

['1443646', 'Booz Allen Hamilton Holding Corp', '10-K', '2013-05-23', 'edgar/data/1443646/0001443646-13-000010.txt', 'edgar/data/1443646/0001443646-13-000010-index.html\r']
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/1443646/000144364613000010/FilingSummary.xml
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364613000010/R1.htm
0001000 - Document - Document and Entity Information
Document and Entity Information
1
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000144364613000010/R2.htm
1001000 - Statement - Consolidated Balance Sheets
Consolidated Balance Sheets
2
------------------------------------------------------------------------------------

['1443646', 'Booz Allen Hamilton Holding Corp', '10-K', '2012-05-30', 'edgar/data/1443646/0001193125-12-253723.txt', 'edgar/data/1443646/0001193125-12-253723-index.html\r']
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/1443646/000119312512253723/FilingSummary.xml
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000119312512253723/R1.htm
00 - Document - Document and Entity Information
Document and Entity Information
1
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1443646/000119312512253723/R2.htm
01 - Statement - Consolidated Balance Sheets
Consolidated Balance Sheets
2
----------------------------------------------------------------------------------------------

In [17]:
#print the name of the documents in a given year
for i in range(len(dfs[0])):
    print(dfs[0][i].index.name)
    
#check if the documents are consistent year-to-year (otherwise might have to make manual adjustment)
for j in range(len(dfs)):
    print(len(dfs[j]))

num_docs = len(dfs[0])
    
    

Consolidated Balance Sheets - USD ($) $ in Thousands
Consolidated Statements of Operations - USD ($) $ in Thousands
Consolidated Statements of Comprehensive Income - USD ($) $ in Thousands
Consolidated Statements of Cash Flows - USD ($) $ in Thousands
4
4
4
4
4
4
4
4
4
4


In [18]:
dfs[0][1].head(10)

,"Mar. 31, 2021"
Consolidated Statements of Operations - USD ($) $ in Thousands,
INCOME STATEMENT [ABSTRACT]-1,NaN
Revenue-1,7858938.0
OPERATING COSTS AND EXPENSES:-2,NaN
Cost of revenue-2,3657530.0
Billable expenses-2,2325888.0
General and administrative expenses-2,1036834.0
Depreciation and amortization-2,84315.0
Total operating costs and expenses-2,7104567.0
Operating income-2,754371.0


In [19]:
dfd = [ele[3] for ele in dfs]
pd.concat(dfd, axis=1)

,"Mar. 31, 2021","Mar. 31, 2020","Mar. 31, 2019","Mar. 31, 2018","Mar. 31, 2017","Mar. 31, 2016","Mar. 31, 2015","Mar. 31, 2014","Mar. 31, 2013","Mar. 31, 2012"
CASH FLOWS FROM OPERATING ACTIVITIES-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Net income-1,608958.0,482603.0,418529.0,305111.0,252490.0,294094.0,232569.0,232188.0,219058.0,239955.0
ADJUSTMENTS TO RECONCILE NET INCOME TO NET CASH PROVIDED BY OPERATING ACTIVITIES:-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Depreciation and amortization-2,84315.0,81081.0,68575.0,64756.0,59544.0,61536.0,62660.0,72327.0,74009.0,75205.0
Noncash lease expense-2,53202.0,55096.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
Escrow payments-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Proceeds from sale of state and local transportation business-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23332.0
Net proceeds from debt-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cash and cash equivalents--beginning of period-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192631.0


In [26]:
num_docs = len(dfs[0])

multi_year_statements = []

for i in range(num_docs):
    year_statements = [ele[i] for ele in dfs]
    multi_year_statements.append(year_statements)
        
    

combined_financial_statements = []    
for i in range(len(multi_year_statements)):
    combined_fs = pd.concat(multi_year_statements[i], axis=1)
    combined_fs.index.name = multi_year_statements[i][0].index.name
    combined_financial_statements.append(combined_fs)

balance_sheet = pd.concat(multi_year_statements[0], axis=1)

combined_financial_statements[3]

,"Mar. 31, 2021","Mar. 31, 2020","Mar. 31, 2019","Mar. 31, 2018","Mar. 31, 2017","Mar. 31, 2016","Mar. 31, 2015","Mar. 31, 2014","Mar. 31, 2013","Mar. 31, 2012"
Consolidated Statements of Cash Flows - USD ($) $ in Thousands,,,,,,,,,,
CASH FLOWS FROM OPERATING ACTIVITIES-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Net income-1,608958.0,482603.0,418529.0,305111.0,252490.0,294094.0,232569.0,232188.0,219058.0,239955.0
ADJUSTMENTS TO RECONCILE NET INCOME TO NET CASH PROVIDED BY OPERATING ACTIVITIES:-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Depreciation and amortization-2,84315.0,81081.0,68575.0,64756.0,59544.0,61536.0,62660.0,72327.0,74009.0,75205.0
Noncash lease expense-2,53202.0,55096.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
Escrow payments-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Proceeds from sale of state and local transportation business-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23332.0
Net proceeds from debt-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
with pd.ExcelWriter("Booz_Allen_finanical_statements.xlsx") as writer:
        combined_financial_statements[0].to_excel(writer, sheet_name='Balance Sheets')
        combined_financial_statements[1].to_excel(writer, sheet_name='Statements of Operations')
        combined_financial_statements[2].to_excel(writer, sheet_name='Comprehensive Income')
        combined_financial_statements[3].to_excel(writer, sheet_name='Statements of Cash Flow')

In [30]:
string = r"Consolidated Statements of Cash Flows - USD ( )  in Thousands"
print(string[0:30])

Consolidated Statements of Cas


In [ ]:
multi_year_statements

In [487]:
print('-'*1000)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Info on the company you want to analyze

In [14]:
#look into master index file for precise spelling
#selectedcompany = 'Booz Allen Hamilton Holding Corp'

selectedcompany = r'Booz'



#select a year you are interested in
selectedyear = 2020

#make sure to put in report exact (10-K vs 10-k)
selectedreport = "10-K"

master_index_file = str(selectedyear) + '-' + find_10k_quarter(selectedcompany, selectedreport, selectedyear) + '.tsv'


#define a user agent for access to the SEC Edgar servers (use browser)
user_agent = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.62"}

print(selectedcompany)
print(master_index_file)


Corsair
2021-QTR1.tsv


### Runs Scrape (In Steps)

In [15]:
# get the index info from downloaded master index
filing_index = get_index_info_on_company(selectedcompany, selectedreport, master_index_file)

['1743759', 'Corsair Gaming, Inc.', '10-K', '2021-03-11', 'edgar/data/1743759/0001564590-21-012507.txt', 'edgar/data/1743759/0001564590-21-012507-index.html\r']


In [16]:
#obtain an xml summary of the file in question
xml_summary = get_xml_summary_of_file(selectedcompany, selectedreport, master_index_file, user_agent)

----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/1743759/000156459021012507/FilingSummary.xml


In [17]:
#generate a master list of sections in the xml file to search
master_reports = parse_xml_summary_of_file(xml_summary, user_agent)

----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1743759/000156459021012507/R1.htm
100000 - Document - Document and Entity Information
Document and Entity Information
1
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1743759/000156459021012507/R2.htm
100010 - Statement - Combined Consolidated Statements of Operations
Combined Consolidated Statements of Operations
2
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1743759/000156459021012507/R3.htm
100020 - Statement - Combined Consolidated Statements of Comprehensive Income (Loss)
Combined Consolidated Statements of Comprehensive Income (Loss)
3
----------------------------------------------------------------------------------------------------
https://www.sec.gov/

In [18]:
statements_data = grab_financial_statements(master_reports)

In [19]:
statements_data

[{'headers': [['Combined Consolidated Statements of Operations - USD ($) shares in Thousands, $ in Thousands',
    '12 Months Ended'],
   ['Dec. 31, 2020', 'Dec. 31, 2019', 'Dec. 31, 2018']],
  'sections': ['INCOME STATEMENT [ABSTRACT]',
   'OPERATING EXPENSES:',
   'OTHER (EXPENSE) INCOME:',
   'NET INCOME (LOSS) PER SHARE',
   'WEIGHTED-AVERAGE SHARES USED TO COMPUTE NET INCOME (LOSS) PER SHARE'],
  'data': [['INCOME STATEMENT [ABSTRACT]', '', '', ''],
   ['Net revenue', '$ 1,702,367', '$ 1,097,174', '$ 937,553'],
   ['Cost of revenue', '1,236,938', '872,887', '744,858'],
   ['Gross profit', '465,429', '224,287', '192,695'],
   ['OPERATING EXPENSES:', '', '', ''],
   ['Sales, general and administrative', '257,004', '163,033', '138,915'],
   ['Product development', '50,064', '37,547', '31,990'],
   ['Total operating expenses', '307,068', '200,580', '170,905'],
   ['Operating income', '158,361', '23,707', '21,790'],
   ['OTHER (EXPENSE) INCOME:', '', '', ''],
   ['Interest expense', '(

In [20]:
df_1 = convert_statements_data_to_pandas(statements_data, 0)
df_1

,"Dec. 31, 2020","Dec. 31, 2019","Dec. 31, 2018"
"Combined Consolidated Statements of Operations - USD ($) shares in Thousands, $ in Thousands",,,
INCOME STATEMENT [ABSTRACT],NaN,NaN,NaN
Net revenue,1702367.0,1097174.0,937553.0
Cost of revenue,1236938.0,872887.0,744858.0
Gross profit,465429.0,224287.0,192695.0
OPERATING EXPENSES:,NaN,NaN,NaN
"Sales, general and administrative",257004.0,163033.0,138915.0
Product development,50064.0,37547.0,31990.0
Total operating expenses,307068.0,200580.0,170905.0
Operating income,158361.0,23707.0,21790.0


In [35]:
print('-'*10000)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# DEBUGGING

## Gather initial index info about company you are interested in

1. Determine the company you are interested in selecting
2. Determine the file you are interested in selecting
3. Go into an index file (e.g. 2019-QTR2.tsv) and search for company name using ctrl-f
4. If file is not quarterly, check if file is located in that quarter 

For the purposes of downloading a master list of sec index information (files downloaded to same directory)

In [16]:
#look into master index file for precise spelling
#selectedcompany = 'Booz Allen Hamilton Holding Corp'

selectedcompany = r'Booz'

#get the CIK number (id for company)
#CIK_number = get_CIK_number(selectedcompany, selectedreport)

#select a year you are interested in
selectedyear = 2021

#make sure to put in report exact (10-K vs 10-k)
selectedreport = "10-K"

master_index_file = str(selectedyear) + '-' + find_10k_quarter(selectedcompany, selectedreport, selectedyear) + '.tsv'


#define a user agent for access to the SEC Edgar servers (use browser)
user_agent = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.62"}

print(selectedcompany)
print(master_index_file)


Booz
2021-QTR2.tsv


In [17]:
#read master index info into a pandas dataframe
master_index = pd.read_csv(master_index_file, sep='\t', lineterminator='\n', names=None)

#rename column for convenience
master_index.columns.values[0] = 'Item'

#select the row in which the company and document you are interested in is located
filing_index = master_index[(master_index['Item'].str.contains(selectedcompany, case=False)) & (master_index['Item'].str.contains(selectedreport, case=False))]

filing_index = filing_index['Item'].str.split('|').to_list()

for elem in filing_index:
    print(elem)
    #if selectedreport in elem:
        #filing_index_specific = elem
        
print('#'*100)

for item in filing_index[0]:
    print(item)
    if '.txt' in item:
         txt_url = item
            


['1443646', 'Booz Allen Hamilton Holding Corp', '10-K', '2021-05-21', 'edgar/data/1443646/0001443646-21-000079.txt', 'edgar/data/1443646/0001443646-21-000079-index.html\r']
####################################################################################################
1443646
Booz Allen Hamilton Holding Corp
10-K
2021-05-21
edgar/data/1443646/0001443646-21-000079.txt
edgar/data/1443646/0001443646-21-000079-index.html


Grab the .txt file

## Grab the Filing XML Summary

In [20]:
#define base url
base_url = r"https://www.sec.gov"

#convert the txt_url to a json url for the document landing page
documents_url = base_url + "/Archives/" + txt_url.replace('-','').replace('.txt', '/index.json')

print(documents_url)

#define user-agent of browser
user_agent = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.62"}


content = requests.get(documents_url, headers=user_agent).json()


#for each file in the document landing page
for file in content['directory']['item']:
    
    #if the filing's name is FilingSummary.xml
    if file['name'] == 'FilingSummary.xml':
        
        #create the url for the xml file
        xml_summary = base_url + content['directory']['name'] + "/" + file['name']
    
        print('-' * 100)
        print('File Name: ' + file['name'])
        print('File Path: ' + xml_summary)


https://www.sec.gov/Archives/edgar/data/1443646/000144364621000079/index.json
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/1443646/000144364621000079/FilingSummary.xml


## Parsing the Filing Summary

In [64]:
#define a new base url that represents the filing folder. This will come in handy when we need to download the reports
base_url = xml_summary.replace('FilingSummary.xml', '')

#request and parse the content (use .content because this is an xml file not json)
content = requests.get(xml_summary, headers=user_agent).content

#create a soup object to parse the content using BeautifulSoup
soup = BeautifulSoup(content, 'lxml')

#find the 'myreports' tag because this contains all the individual reports submitted
reports = soup.find('myreports')

reports

<myreports>
<report instance="zbh-20151231.xml">
<isdefault>false</isdefault>
<hasembeddedreports>false</hasembeddedreports>
<htmlfilename>R1.htm</htmlfilename>
<longname>1001 - Document - Document and Entity Information</longname>
<reporttype>Sheet</reporttype>
<role>http://www.zimmer.com/taxonomy/role/DocumentandEntityInformation</role>
<shortname>Document and Entity Information</shortname>
<menucategory>Cover</menucategory>
<position>1</position>
</report>
<report instance="zbh-20151231.xml">
<isdefault>false</isdefault>
<hasembeddedreports>false</hasembeddedreports>
<htmlfilename>R2.htm</htmlfilename>
<longname>1003 - Statement - Consolidated Statements of Earnings</longname>
<reporttype>Sheet</reporttype>
<role>http://www.zimmer.com/taxonomy/role/StatementOfIncome</role>
<shortname>Consolidated Statements of Earnings</shortname>
<menucategory>Statements</menucategory>
<position>2</position>
</report>
<report instance="zbh-20151231.xml">
<isdefault>false</isdefault>
<hasembeddedrep

In [65]:
#define a new base url that represents the filing folder. This will come in handy when we need to download the reports
base_url = xml_summary.replace('FilingSummary.xml', '')

#request and parse the content (use .content because this is an xml file not json)
content = requests.get(xml_summary, headers=user_agent).content

#create a soup object to parse the content using BeautifulSoup
soup = BeautifulSoup(content, 'lxml')

#find the 'myreports' tag because this contains all the individual reports submitted
reports = soup.find('myreports')

#want a list ot store all theindividual components of the report, so create the master list
master_reports = []

# loop through each report in the 'myreports' tag but avoid the last one as this will cause an error
# (last one has differnent formatting).
for index, report in enumerate(reports.find_all('report')[:-1]):
    
    #create a dictionary to store all the differnet parts we need
    report_dict = {}
    report_dict['name_short'] = report.shortname.text
    report_dict['name_long'] = report.longname.text
    report_dict['position'] = index+1
    #report_dict['category'] = report.menucategory.text
    report_dict['url'] = base_url + report.htmlfilename.text
    
    #append the dictionary to the master list.
    master_reports.append(report_dict)
    
    #print the info to the user.
    print('-'*100)
    print(base_url + report.htmlfilename.text)
    print(report.longname.text)
    print(report.shortname.text)
    #print(report.menucategory.text)
    print(index+1)

----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1136869/000119312516485039/R1.htm
1001 - Document - Document and Entity Information
Document and Entity Information
1
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1136869/000119312516485039/R2.htm
1003 - Statement - Consolidated Statements of Earnings
Consolidated Statements of Earnings
2
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1136869/000119312516485039/R3.htm
1004 - Statement - Consolidated Statements of Comprehensive (Loss) Income
Consolidated Statements of Comprehensive (Loss) Income
3
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1136869/000119312516485039

## Grabbing the Financial Statements

1. Balance Sheet
2. Statement of Cash Flows
3. Income Statement
4. Statement of Stock Holder Equity

In [66]:

# TO-DO: Figure out how to standardize getting the consolidated financial statements because they are not the same between
#        Booz Allen and the company Sigma Coding analyzed

#####################################################################################

#create the list to hold the statement urls
statements_url = []

for report_dict in master_reports:
    # define the statements we want to look for.
    
    item1 = r"CONSOLIDATED"
    item2 = r"PARENTHETICAL"
    
    if item1 in report_dict['name_short'].upper() and item2 not in report_dict['name_short'].upper() and report_dict['position'] < 11:
        print(report_dict['position'])
        
        statements_url.append(report_dict['url'])
        
        



2
3
4
6
7


## Scraping the Financial Statements

In [67]:
#assuming we want all the statements in a single data set
statements_data = []

#loop through each statement url
for statement in statements_url:
    
    #define a dictionary that will store the different parts of the statement
    statement_data = {}
    statement_data['headers'] = []
    statement_data['sections'] = []
    statement_data['data'] = []
    
    #request the statement file content
    content = requests.get(statement, headers=user_agent).content
    report_soup = BeautifulSoup(content, 'html')
    
    #find all the rows, figure out what type of row it is, parse the elements, and store in teh statement file list.
    #tr = table row
    for index, row in enumerate(report_soup.table.find_all('tr')):
        
        #first let's get all the elements (td = standard data cell)
        # cols represents elements in the row
        cols = row.find_all('td')
            
        # if it's a regular row and not a section or a table header (th = header, strong = important)
        if (len(row.find_all('th')) == 0 and len(row.find_all('table')) == 0): # and len(row.find_all('strong')) == 0:
            reg_row = []
            for ele in cols:
                
                if len(row.find_all('strong')) != 0:
                    reg_row.append(ele.text.strip().upper())
                else:
                    reg_row.append(ele.text.strip())
            
            #check if it is a section label and store that it in sec_row
            if len(row.find_all('strong')) != 0:
                sec_row = cols[0].text.strip().upper()
                statement_data['sections'].append(sec_row)
                    
    
                
            #reg_row = [ele.text.strip() for ele in cols]
            statement_data['data'].append(reg_row)
        
        #if it's a regular row and a section but not a table header
        #elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
            #sec_row = cols[0].text.strip().upper()
            #statement_data['sections'].append(sec_row)
            
        #finally if it's not any of those it must be a header
        elif (len(row.find_all('th')) != 0):
            hed_row = []
            for ele in row.find_all('th'):
                if (len(ele.find_all('sup'))) == 0:
                    hed_row.append(ele.text.strip())
            statement_data['headers'].append(hed_row)
            
        else:
            print("We encountered an error")
    
    statements_data.append(statement_data)
    
    

## Converting the Data into a Data Frame

Let's do the Balance Sheet

In [68]:
document_num = 0
print(statements_data[document_num]['data'])

[['INCOME STATEMENT [ABSTRACT]', '', '', ''], ['Net Sales', '$ 5,997.8', '$ 4,673.3', '$ 4,623.4'], ['Cost of products sold, excluding intangible asset amortization', '1,800.6', '1,242.8', '1,266.7'], ['Intangible asset amortization', '337.4', '92.5', '78.5'], ['Research and development', '268.8', '187.4', '203.0'], ['Selling, general and administrative', '2,284.2', '1,750.7', '1,749.3'], ['Certain claims (Note 20)', '7.7', '21.5', '47.0'], ['Special items (Note 3)', '831.8', '341.1', '210.3'], ['Operating expenses', '5,530.5', '3,636.0', '3,554.8'], ['Operating Profit', '467.3', '1,037.3', '1,068.6'], ['Other expense, net', '(36.9)', '(46.7)', '(6.0)'], ['Interest income', '9.4', '11.9', '15.6'], ['Interest expense', '(286.6)', '(63.1)', '(70.1)'], ['Earnings before income taxes', '153.2', '939.4', '1,008.1'], ['Provision for income taxes', '7.0', '220.2', '229.5'], ['Net earnings', '146.2', '719.2', '778.6'], ['Less: Net loss attributable to noncontrolling interest', '(0.8)', '(1.1)'

In [69]:
#define what document you want to convert (0 to n documents)
document_num = 3


#balance_header = statements_data[0]['headers'][-1]
balance_data = statements_data[document_num]['data']

df = pd.DataFrame(balance_data)


#Define the Index column, rename it, and we need to make sure to drop the old column once we reindex
df.index = df[0]
#df.index.name = 'Category'
df = df.drop(0, axis = 1)


# Get rid of the '$', '(', ')', and convert the '' to NaNs.
df = df.replace('[\$,)]','', regex=True )\
                     .replace( '[(]','-', regex=True)\
                     .replace( '', 'NaN', regex=True)


# Display
print('-'*100)
print('Before type conversion')
print('-'*100)
print(statements_data[document_num]['headers'])
display(df)


----------------------------------------------------------------------------------------------------
Before type conversion
----------------------------------------------------------------------------------------------------
[["Consolidated Statements of Stockholders' Equity - USD ($) shares in Millions, $ in Millions", 'Total', 'Common Shares [Member]', 'Paid-in Capital [Member]', 'Retained Earnings [Member]', 'Accumulated Other Comprehensive (Loss) Income [Member]', 'Treasury Shares [Member]', 'Noncontrolling Interest [Member]']]


,1,2,3,4,5,6,7
0,,,,,,,
"Balance at Dec. 31, 2012",5848.0,2.6,3500.6,7143.2,268.3,-5072.1,5.4
"Balance, shares at Dec. 31, 2012",NaN,257.1,NaN,NaN,NaN,-85.5,NaN
Net earnings,778.6,NaN,NaN,780.4,NaN,NaN,-1.8
Other comprehensive income (loss),32.4,NaN,NaN,NaN,32.6,NaN,-0.2
Purchase of additional shares from noncontrolling interest,-1.7,NaN,-1.1,NaN,NaN,NaN,-0.6
Cash dividends declared,-135.4,NaN,NaN,-135.4,NaN,NaN,NaN
"Stock compensation plans, including tax benefits",507.7,NaN,501.1,1.2,NaN,5.4,NaN
"Stock compensation plans, including tax benefits, shares",NaN,7.2,NaN,NaN,NaN,0.1,NaN
Share repurchases,-719.0,NaN,NaN,NaN,NaN,-719.0,NaN


In [70]:
    
for i in range(df.shape[1]):
    df[df.columns[i]] = df[df.columns[i]].apply(change_to_float)    
df = df.astype(float)
df = get_rid_of_null_columns(df)
display(df)
print(df.shape)

,1,2,3,4,5,6,7
0,,,,,,,
"Balance at Dec. 31, 2012",58480.0,26.0,35006.0,71432.0,2683.0,-50721.0,54.0
"Balance, shares at Dec. 31, 2012",NaN,2571.0,NaN,NaN,NaN,-855.0,NaN
Net earnings,7786.0,NaN,NaN,7804.0,NaN,NaN,-18.0
Other comprehensive income (loss),324.0,NaN,NaN,NaN,326.0,NaN,-2.0
Purchase of additional shares from noncontrolling interest,-17.0,NaN,-11.0,NaN,NaN,NaN,-6.0
Cash dividends declared,-1354.0,NaN,NaN,-1354.0,NaN,NaN,NaN
"Stock compensation plans, including tax benefits",5077.0,NaN,5011.0,12.0,NaN,54.0,NaN
"Stock compensation plans, including tax benefits, shares",NaN,72.0,NaN,NaN,NaN,1.0,NaN
Share repurchases,-7190.0,NaN,NaN,NaN,NaN,-7190.0,NaN


(32, 7)


In [71]:
#below is to account for the headers having multiple levels versus only one level
if len(statements_data[document_num]['headers']) == 1:
    header = statements_data[document_num]['headers'][0][1:]
    document = statements_data[document_num]['headers'][0][0]
    
elif len(statements_data[document_num]['headers']) == 2:
    header = statements_data[document_num]['headers'][-1]
    document = statements_data[document_num]['headers'][0][0] 
else:
    header = df.columns
    document = 'Category'
    
    
    
print(header)
print(document)

['Total', 'Common Shares [Member]', 'Paid-in Capital [Member]', 'Retained Earnings [Member]', 'Accumulated Other Comprehensive (Loss) Income [Member]', 'Treasury Shares [Member]', 'Noncontrolling Interest [Member]']
Consolidated Statements of Stockholders' Equity - USD ($) shares in Millions, $ in Millions


In [72]:
# everything is a string, so let's convert all the data to a float
df = df.astype(float)

#Change the column headers 
df.columns = header

#Change the name of the index
df.index.name = document

#Display
print('-'*100)
print('Final Product')
print('-'*100)

#show the df
df

----------------------------------------------------------------------------------------------------
Final Product
----------------------------------------------------------------------------------------------------


,Total,Common Shares [Member],Paid-in Capital [Member],Retained Earnings [Member],Accumulated Other Comprehensive (Loss) Income [Member],Treasury Shares [Member],Noncontrolling Interest [Member]
"Consolidated Statements of Stockholders' Equity - USD ($) shares in Millions, $ in Millions",,,,,,,
"Balance at Dec. 31, 2012",58480.0,26.0,35006.0,71432.0,2683.0,-50721.0,54.0
"Balance, shares at Dec. 31, 2012",NaN,2571.0,NaN,NaN,NaN,-855.0,NaN
Net earnings,7786.0,NaN,NaN,7804.0,NaN,NaN,-18.0
Other comprehensive income (loss),324.0,NaN,NaN,NaN,326.0,NaN,-2.0
Purchase of additional shares from noncontrolling interest,-17.0,NaN,-11.0,NaN,NaN,NaN,-6.0
Cash dividends declared,-1354.0,NaN,NaN,-1354.0,NaN,NaN,NaN
"Stock compensation plans, including tax benefits",5077.0,NaN,5011.0,12.0,NaN,54.0,NaN
"Stock compensation plans, including tax benefits, shares",NaN,72.0,NaN,NaN,NaN,1.0,NaN
Share repurchases,-7190.0,NaN,NaN,NaN,NaN,-7190.0,NaN


In [73]:
print(statements_data[document_num]['sections'])

[]


In [74]:
sections = statements_data[document_num]['sections']

for ele in df.index:
    if ele in sections:
        print('True')

df4 = add_tags_for_sections(statements_data[document_num]['sections'], df.copy())
#print(statements_data[document_num]['sections'])
#print(df.index)

df4

,Total,Common Shares [Member],Paid-in Capital [Member],Retained Earnings [Member],Accumulated Other Comprehensive (Loss) Income [Member],Treasury Shares [Member],Noncontrolling Interest [Member]
"Consolidated Statements of Stockholders' Equity - USD ($) shares in Millions, $ in Millions",,,,,,,
"Balance at Dec. 31, 2012",58480.0,26.0,35006.0,71432.0,2683.0,-50721.0,54.0
"Balance, shares at Dec. 31, 2012",NaN,2571.0,NaN,NaN,NaN,-855.0,NaN
Net earnings,7786.0,NaN,NaN,7804.0,NaN,NaN,-18.0
Other comprehensive income (loss),324.0,NaN,NaN,NaN,326.0,NaN,-2.0
Purchase of additional shares from noncontrolling interest,-17.0,NaN,-11.0,NaN,NaN,NaN,-6.0
Cash dividends declared,-1354.0,NaN,NaN,-1354.0,NaN,NaN,NaN
"Stock compensation plans, including tax benefits",5077.0,NaN,5011.0,12.0,NaN,54.0,NaN
"Stock compensation plans, including tax benefits, shares",NaN,72.0,NaN,NaN,NaN,1.0,NaN
Share repurchases,-7190.0,NaN,NaN,NaN,NaN,-7190.0,NaN


In [147]:
df.shape

(33, 2)

In [148]:
df3 = add_tags_to_repeating_indices(df.copy(), '@')
df3

,"Mar. 31, 2020","Mar. 31, 2019"
Consolidated Balance Sheets - USD ($) $ in Thousands,,
CURRENT ASSETS:,NaN,NaN
Cash and cash equivalents,741901.0,283990.0
"Accounts receivable, net of allowance",1459471.0,1330364.0
Prepaid expenses and other current assets,126816.0,84986.0
Total current assets,2328188.0,1699340.0
"Property and equipment, net of accumulated depreciation",208077.0,172453.0
Operating lease right-of-use assets,240122.0,0.0
"Intangible assets, net of accumulated amortization",300987.0,287051.0
Goodwill,1581160.0,1581160.0


In [149]:
## Purpose: Some indices repeat in a financial statement (e.g. others). To make sure each index is unique without
##          losing information in the dataframe, we can add a tag to the end. The number of tags at the end represent
##          the number of times that index value has been seen before (no tag means first time, one tag means second, two
##          tags means third)
def add_tags_to_repeating_indices(df, tag):
    name = df.index.name
    idx = df.index.to_list()
    dup = {x for x in idx if idx.count(x) > 1}

    for ele in dup:
        end = ''
        for i in range(len(idx)):
            if ele == idx[i]:
                idx[i] = idx[i] + end
                end = end + tag
            
    df.index = idx
    df.index.name = name
    return df



In [5]:
# Purpose: Obtains the CIK number pertaining to a specified company.

def get_CIK_number(selected_company):
    master_index_file = '2020-QTR1.tsv'
    
    
    master_index = pd.read_csv(master_index_file, sep='\t', lineterminator='\n', names=None)
        
    master_index.columns.values[0] = 'Item'
    
    filing_index = master_index[(master_index['Item'].str.contains(selected_company, case=False))]
    
    return filing_index.iloc[0, 0].split('|')[0]